In [2]:
import os
from os.path import join
import glob
import pandas as pd
import numpy as np
import time
import seaborn as sns
import matplotlib.pyplot as plt
%load_ext autoreload

In [1]:
def display_all(df):
    with pd.option_context("display.max_rows", 100, "display.max_columns", 100): 
        print(df)

In [3]:

def make_value2index(attacks):
    #make dictionary
    attacks = sorted(attacks)
    d = {}
    counter=0
    for attack in attacks:
        d[attack] = counter
        counter+=1
    return d

In [4]:

# chganges label from string to integer/index
def encode_label(Y_str):
    labels_d = make_value2index(np.unique(Y_str))
    Y = [labels_d[y_str] for y_str  in Y_str]
    Y = np.array(Y)
    return np.array(Y)

In [5]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/Colab Notebooks/NetworkAnomalyDetectionML/")

Mounted at /content/drive


In [6]:
# All columns
col_names = np.array(['Source Port', 'Destination Port',
                      'Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets',
                      'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean',
                      'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std',
                      'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total',
                      'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max',
                      'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Fwd Header Length', 'Bwd Header Length',
                      'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std',
                      'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
                      'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size',
                      'Avg Bwd Segment Size','Subflow Fwd Packets', 'Subflow Fwd Bytes',
                      'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward',
                      'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean',
                      'Idle Std', 'Idle Max', 'Idle Min', 'Label'])

In [7]:
col_names.shape

(72,)

In [19]:
df_train = pd.read_csv('./train_set.csv',names=col_names, skiprows=1)
df_train = df_train.replace([-np.inf, np.inf], np.nan)
df_train = df_train.dropna()

In [20]:
print(df_train.shape)

(556464, 72)


In [21]:
df_train['Label'].value_counts()

BENIGN                        278186
DoS Hulk                      115062
PortScan                       79403
DDoS                           64013
DoS GoldenEye                   5146
FTP-Patator                     3968
SSH-Patator                     2948
DoS slowloris                   2898
DoS Slowhttptest                2749
Bot                              979
Web Attack  Brute Force         753
Web Attack  XSS                 326
Infiltration                      18
Web Attack  Sql Injection        10
Heartbleed                         5
Name: Label, dtype: int64

In [22]:
df_test = pd.read_csv('./test_set.csv',names=col_names, skiprows=1)  
print('Test set size: ', df_test.shape)
df_test = df_test.replace([-np.inf, np.inf], np.nan)
df_test= df_test.dropna()

df_val = pd.read_csv('./crossval_set.csv',names=col_names, skiprows=1)  
df_val = df_val.replace([-np.inf, np.inf], np.nan)
df_val = df_val.dropna()
print('Validation set size: ', df_val.shape)

Test set size:  (278338, 72)
Validation set size:  (278208, 72)


In [23]:
df_label = df_train['Label']
data = df_train.drop(columns=['Label'])
X_train = data.values
y_train = encode_label(df_label.values)

In [24]:
from sklearn.preprocessing import MinMaxScaler

In [25]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_train

array([[0.        , 0.01218679, 0.02201756, ..., 0.        , 0.6925    ,
        0.6925    ],
       [0.00261666, 0.00100996, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.00925792, 0.01911093, ..., 0.        , 0.82      ,
        0.82      ],
       [0.00261666, 0.00100996, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.00260905, 0.0031095 , ..., 0.        , 0.        ,
        0.        ]])

In [26]:
df_label = df_test['Label']
data = df_test.drop(columns=['Label'])
X_test = data.values
y_test = encode_label(df_label.values)

In [27]:

scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test)
X_test

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 2.97173782e-02, 3.33556525e-02, ...,
        0.00000000e+00, 1.33333333e-01, 1.33333333e-01],
       [2.32445521e-02, 8.08177992e-03, 0.00000000e+00, ...,
        0.00000000e+00, 5.57552225e-01, 5.57552225e-01],
       ...,
       [9.68523002e-04, 3.36740830e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 9.65323712e-03, 2.27123352e-02, ...,
        0.00000000e+00, 6.90833333e-01, 6.90833333e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [28]:
df_label = df_val['Label']
data = df_val.drop(columns=['Label'])
X_val = data.values
y_val = encode_label(df_label.values)

In [29]:
scaler = MinMaxScaler()
X_val = scaler.fit_transform(X_val)
X_val

array([[0.0188862 , 0.00761064, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00290557, 0.00117087, 0.        , ..., 0.        , 0.06068153,
        0.06068153],
       [0.02324455, 0.00936694, 0.        , ..., 0.        , 0.57472972,
        0.57472972],
       ...,
       [0.02663438, 0.01073295, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00290557, 0.00117087, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01694915, 0.00683006, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [30]:
from tensorflow.keras.utils import to_categorical

In [31]:
y_train = to_categorical(y_train, 15)
y_test = to_categorical(y_test, 15)
y_val = to_categorical(y_val, 15)

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, Flatten, Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm

In [33]:
#hyper-params
batch_size = 1024 # increasing batch size with more gpu added
input_dim = X_train.shape[1]
num_class = 15                   # 15 intrusion classes, including benign traffic class
num_epochs = 30
learning_rates = 1e-3
regularizations = 1e-3
optim = tf.keras.optimizers.Adam(lr=learning_rates, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

print(input_dim)
print(num_class)

71
15


In [34]:
X_train_r = np.zeros((len(X_train), input_dim, 1))
X_train_r[:, :, 0] = X_train[:, :input_dim]
print(X_train_r.shape)

(556464, 71, 1)


In [35]:
X_test_r = np.zeros((len(X_test), input_dim, 1))
X_test_r[:, :, 0] = X_test[:, :input_dim]
print(X_test_r.shape)

(278192, 71, 1)


In [36]:
X_val_r = np.zeros((len(X_val), input_dim, 1))
X_val_r[:, :, 0] = X_val[:, :input_dim]
print(X_val_r.shape)

(278208, 71, 1)


In [37]:
model = Sequential()

# input layer
model.add(Conv1D(filters=64, kernel_size=3, padding='same', input_shape=(71,1)))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))


model.add(Conv1D(filters=128, kernel_size=3))
model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))

model.add(Flatten())
#model.add(Dense(100, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(num_class))
model.add(Activation('softmax'))


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 71, 64)            256       
_________________________________________________________________
batch_normalization (BatchNo (None, 71, 64)            284       
_________________________________________________________________
activation (Activation)      (None, 71, 64)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 69, 128)           24704     
_________________________________________________________________
batch_normalization_1 (Batch (None, 69, 128)           276       
_________________________________________________________________
activation_1 (Activation)    (None, 69, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 8832)              0

In [38]:
model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])

In [39]:
model.fit(X_train_r, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_val_r, y_val), verbose=1)

Epoch 1/30
544/544 [==============================] - 396s 726ms/step - loss: 0.3850 - accuracy: 0.8857 - val_loss: 0.1940 - val_accuracy: 0.9343
Epoch 2/30
544/544 [==============================] - 398s 731ms/step - loss: 0.1054 - accuracy: 0.9649 - val_loss: 0.0934 - val_accuracy: 0.9677
Epoch 3/30
544/544 [==============================] - 396s 728ms/step - loss: 0.0915 - accuracy: 0.9679 - val_loss: 0.0801 - val_accuracy: 0.9727
Epoch 4/30
544/544 [==============================] - 396s 728ms/step - loss: 0.0757 - accuracy: 0.9744 - val_loss: 0.0630 - val_accuracy: 0.9784
Epoch 5/30
544/544 [==============================] - 397s 730ms/step - loss: 0.0653 - accuracy: 0.9779 - val_loss: 0.0578 - val_accuracy: 0.9806
Epoch 6/30
544/544 [==============================] - 398s 731ms/step - loss: 0.0607 - accuracy: 0.9797 - val_loss: 0.0597 - val_accuracy: 0.9792
Epoch 7/30
544/544 [==============================] - 396s 729ms/step - loss: 0.0579 - accuracy: 0.9804 - val_loss: 0.0531 -

In [40]:
accuracy = model.evaluate(X_test_r, y_test, batch_size=batch_size, verbose=0)

In [41]:
model.save('./cicids2017cnn.h5')

In [42]:
weights_list = model.get_weights()
i = 0
for item in weights_list:
    i=i+1
    print("Layer " + str(i))
    if (i == 15):
        fc = item.squeeze()

print(fc)

Layer 1
Layer 2
Layer 3
Layer 4
Layer 5
Layer 6
Layer 7
Layer 8
Layer 9
Layer 10
Layer 11
Layer 12
Layer 13
Layer 14
Layer 15
Layer 16
[[-3.49025190e-01  2.58190304e-01  1.14115849e-01  1.21773337e-03
  -1.02339022e-01 -1.87414959e-01  2.15336904e-01 -9.92359146e-02
   3.86462122e-01 -1.13670908e-01  1.53055087e-01 -1.02035664e-01
  -1.72357976e-01  1.82847723e-01  3.12387139e-01]
 [-6.47033602e-02  2.46246830e-02 -2.83273160e-01 -3.69765639e-01
  -3.17917645e-01 -9.19328332e-02 -3.39699417e-01  1.90721713e-02
  -9.49534923e-02  6.46063611e-02 -3.26794952e-01  3.69666010e-01
  -2.42836043e-01 -2.56030262e-01  3.29432875e-01]
 [ 2.13969499e-01 -4.54774424e-02 -3.26026708e-01 -1.58795863e-01
   6.74697608e-02  7.98517317e-02  4.33548875e-02  3.59223895e-02
   3.52827519e-01  2.93886125e-01  3.43185365e-01  8.16893652e-02
  -1.16575144e-01  7.62326494e-02  3.12474631e-02]
 [-3.86507303e-01 -1.24521054e-01 -6.27443910e-01 -9.27245378e-01
  -7.55071163e-01  2.13509053e-01  1.60530791e-01 -1

In [43]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [44]:
df_label = df_test['Label']
ytest = encode_label(df_label.values)


In [45]:

# predict probabilities for test set
yhat_probs = model.predict(X_test_r, verbose=0)

# predict crisp classes for test set
yhat_classes = model.predict_classes(X_test_r, verbose=0)

# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
#yhat_classes = yhat_classes[:, 0]

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [46]:
accuracy = accuracy_score(ytest, yhat_classes)
print('Accuracy: %f' % accuracy)

Accuracy: 0.994011


In [47]:
precision = precision_score(ytest, yhat_classes, labels=[1,2], average='micro')
# labels is a list of all possible class labels
print('Precision: %f' % precision)

Precision: 0.993992


In [48]:
recall = recall_score(ytest, yhat_classes, average='weighted')
print('Recall: %f' % recall)

Recall: 0.994011


In [49]:
f1 = f1_score(ytest, yhat_classes, average='weighted')
print('F1 score: %f' % f1)

F1 score: 0.993871


In [50]:
kappa = cohen_kappa_score(ytest, yhat_classes)
print('Cohens kappa: %f' % kappa)

# confusion matrix
matrix = confusion_matrix(ytest, yhat_classes)
print(matrix)

Cohens kappa: 0.991116
[[138121    146     34      6    567     39      2     10      0      2
      68     67      1      0      1]
 [    58    429      0      0      0      0      0      0      0      0
       0      0      0      0      0]
 [     7      0  31996      0      0      2      0      0      0      0
       0      0      0      0      0]
 [   105      0      0   2388     23      2      0      0      0      0
       5      0      0     50      0]
 [   125      0      1      0  57403      0      0      0      0      0
       2      0      0      0      0]
 [    11      0      0      0      1   1356      3      0      0      0
       0      0      3      0      0]
 [     6      0      0      0      0     15   1427      0      0      0
       0      0      1      0      0]
 [    28      0      0      0      0      0      0   1954      0      0
       0      0      1      0      0]
 [     0      0      0      0      0      0      0      0      2      0
       0      0      0   